In [2]:
# hide warnings to keep things tidy.
import warnings
warnings.filterwarnings('ignore')
import random

import numpy as np

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
matplotlib.style.use('ggplot') # make things a bit prettier.

import pandas as pd
import statsmodels.api as sm 
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsRegressor
import random
from numpy.random import permutation
import math
from sklearn import metrics
df = pd.read_csv("/Users/sushma/Desktop/DM_Git/Recommender_System/FirstRatingsMatrix.csv", header=0)
df.head()

,UserID,MovieID,Rating,label
0,1,661,3,0
1,1,914,3,0
2,1,594,4,0
3,1,938,4,0
4,1,2687,3,0


In [33]:
uniq_userid = df.UserID.unique()
sorted_uniq_userid = sorted(uniq_userid)
uindex = 0
user_index = {}
for n in sorted_uniq_userid:
    user_index[n] = uindex
    uindex += 1
print user_index[77]
uniq_movieid = df.MovieID.unique()
sorted_uniq_movieid = sorted(uniq_movieid)
mindex = 0
movie_index = {}
for n in sorted_uniq_movieid:
    movie_index[n] = mindex
    mindex += 1
print movie_index[77]

76
20


In [11]:
n_users = df.UserID.unique().shape[0]
print("n users: %s" % n_users)
n_movies = df.MovieID.unique().shape[0]
print("n movies: %s" % n_movies)

n users: 6037
n movies: 727


In [47]:
first_clust_ratings = np.zeros((n_users,n_movies))
for row in df.itertuples():
    # row[1] will the user_id; row[2] the 
    # rating (see also my note below).
    # the -1 2s to move to 0 indexing.
    first_clust_ratings[user_index[row[1]]-1, movie_index[row[2]]-1] = row[3]
first_clust_ratings

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [48]:
sparsity = float(len(first_clust_ratings.nonzero()[0]))
sparsity /= (first_clust_ratings.shape[0] * first_clust_ratings.shape[1])
sparsity *= 100
print('sparsity: {:4.2f}%'.format(sparsity))

sparsity: 5.18%


In [17]:
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()
    for user in range(ratings.shape[0]):
        if len(ratings[user, :].nonzero()[0]) > 5:
            # sample 6 ratings from each user to use
            # as 'test' data
            test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                                size=6, 
                                                replace=False)
            # effectively remove these from the training
            # set
            train[user, test_ratings] = 0.
            test[user, test_ratings] = ratings[user, test_ratings]
           
    # make sure that test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    
    return train, test

In [78]:
train, test = train_test_split(first_clust_ratings)

In [20]:
def similarity(ratings, kind='user', epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    if kind == 'user':
        sim = ratings.dot(ratings.T) + epsilon
        print(sim)
    elif kind == 'item':
        # we need only flip the dimensions around 
        # to do item based similarity!
        sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / (norms * norms.T))

In [80]:
user_similarity = similarity(train, kind='user')
item_similarity = similarity(train, kind='item')

[[  2.95000000e+02   4.10000000e+01   1.00000000e-09 ...,   3.60000000e+01
    5.70000000e+01   1.00000000e-09]
 [  4.10000000e+01   9.80000000e+01   1.00000000e-09 ...,   1.00000000e-09
    3.10000000e+01   1.00000000e-09]
 [  1.00000000e-09   1.00000000e-09   2.50000000e+01 ...,   1.00000000e-09
    2.00000000e+01   1.00000000e-09]
 ..., 
 [  3.60000000e+01   1.00000000e-09   1.00000000e-09 ...,   3.71000000e+02
    8.60000000e+01   1.60000000e+01]
 [  5.70000000e+01   3.10000000e+01   2.00000000e+01 ...,   8.60000000e+01
    6.76000000e+02   1.00000000e-09]
 [  1.00000000e-09   1.00000000e-09   1.00000000e-09 ...,   1.60000000e+01
    1.00000000e-09   7.30000000e+01]]


In [82]:
sim = first_clust_ratings.dot(train.T) + 1e-9
sim

array([[  2.95000000e+02,   6.10000000e+01,   1.00000000e-09, ...,
          3.60000000e+01,   5.70000000e+01,   1.00000000e-09],
       [  7.70000000e+01,   9.80000000e+01,   1.00000000e-09, ...,
          2.00000000e+01,   6.60000000e+01,   1.00000000e-09],
       [  3.20000000e+01,   1.60000000e+01,   2.50000000e+01, ...,
          1.50000000e+01,   5.30000000e+01,   1.00000000e-09],
       ..., 
       [  3.60000000e+01,   1.00000000e-09,   1.00000000e-09, ...,
          3.71000000e+02,   1.22000000e+02,   1.60000000e+01],
       [  5.70000000e+01,   3.10000000e+01,   2.00000000e+01, ...,
          8.60000000e+01,   6.76000000e+02,   1.00000000e-09],
       [  1.00000000e-09,   1.00000000e-09,   2.00000000e+01, ...,
          3.70000000e+01,   1.60000000e+01,   7.30000000e+01]])

In [24]:
def predict(ratings, similarity, kind='user'):
    if kind == 'user':
        return similarity.dot(ratings) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif kind == 'item':
        return ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

In [25]:
from sklearn.metrics import mean_squared_error
def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [27]:
item_prediction = predict(train, item_similarity, kind='item')
user_prediction = predict(train, user_similarity, kind='user')
print('User-based CF MSE: ' + str(get_mse(user_prediction, test)))
print('Movie-based CF MSE: ' + str(get_mse(item_prediction, test)))

User-based CF MSE: 9.12508205207
Movie-based CF MSE: 12.0897997254


In [34]:
df_second = pd.read_csv("/Users/sushma/Desktop/DM_Git/Recommender_System/SecondRatingsMatrix.csv", header=0)
df_second.head()

,UserID,MovieID,Rating,label
0,1,2918,4,1
1,1,2321,3,1
2,1,1270,5,1
3,1,1566,4,1
4,1,588,4,1


In [35]:
uniq_userid = df_second.UserID.unique()
sorted_uniq_userid = sorted(uniq_userid)
uindex = 0
user_index = {}
for n in sorted_uniq_userid:
    user_index[n] = uindex
    uindex += 1
uniq_movieid = df_second.MovieID.unique()
sorted_uniq_movieid = sorted(uniq_movieid)
mindex = 0
movie_index = {}
for n in sorted_uniq_movieid:
    movie_index[n] = mindex
    mindex += 1

In [36]:
n_users = df_second.UserID.unique().shape[0]
print("n users: %s" % n_users)
n_movies = df_second.MovieID.unique().shape[0]
print("n movies: %s" % n_movies)

n users: 5963
n movies: 558


In [37]:
second_clust_ratings = np.zeros((n_users,n_movies))
for row in df_second.itertuples():
    # row[1] will the user_id; row[2] the 
    # rating (see also my note below).
    # the -1 2s to move to 0 indexing.
    second_clust_ratings[user_index[row[1]]-1, movie_index[row[2]]-1] = row[3]
second_clust_ratings

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  3.],
       [ 0.,  0.,  0., ...,  0.,  0.,  5.]])

In [38]:
sparsity = float(len(second_clust_ratings.nonzero()[0]))
sparsity /= (second_clust_ratings.shape[0] * second_clust_ratings.shape[1])
sparsity *= 100
print('sparsity: {:4.2f}%'.format(sparsity))

sparsity: 5.60%


In [39]:
train, test = train_test_split(second_clust_ratings)

In [40]:
user_similarity = similarity(train, kind='user')
item_similarity = similarity(train, kind='item')

[[  6.90000000e+01   6.00000000e+00   5.00000000e+00 ...,   9.00000000e+00
    3.50000000e+01   1.00000000e-09]
 [  6.00000000e+00   1.21000000e+02   1.00000000e-09 ...,   3.80000000e+01
    4.10000000e+01   1.00000000e-09]
 [  5.00000000e+00   1.00000000e-09   3.07000000e+02 ...,   1.00000000e-09
    6.50000000e+01   1.00000000e-09]
 ..., 
 [  9.00000000e+00   3.80000000e+01   1.00000000e-09 ...,   3.25000000e+02
    6.90000000e+01   1.60000000e+01]
 [  3.50000000e+01   4.10000000e+01   6.50000000e+01 ...,   6.90000000e+01
    4.90000000e+02   1.00000000e-09]
 [  1.00000000e-09   1.00000000e-09   1.00000000e-09 ...,   1.60000000e+01
    1.00000000e-09   1.60000000e+01]]


In [41]:
sim = second_clust_ratings.dot(train.T) + 1e-9
sim

array([[  6.90000000e+01,   6.00000000e+00,   1.40000000e+01, ...,
          2.50000000e+01,   5.10000000e+01,   1.00000000e-09],
       [  6.00000000e+00,   1.21000000e+02,   2.70000000e+01, ...,
          3.80000000e+01,   7.30000000e+01,   1.00000000e-09],
       [  8.00000000e+00,   1.00000000e-09,   3.07000000e+02, ...,
          1.00000000e-09,   8.40000000e+01,   1.00000000e-09],
       ..., 
       [  9.00000000e+00,   4.60000000e+01,   1.00000000e-09, ...,
          3.25000000e+02,   1.00000000e+02,   1.60000000e+01],
       [  3.50000000e+01,   5.10000000e+01,   6.50000000e+01, ...,
          6.90000000e+01,   4.90000000e+02,   1.00000000e-09],
       [  9.00000000e+00,   1.50000000e+01,   1.00000000e-09, ...,
          3.60000000e+01,   3.00000000e+01,   1.60000000e+01]])

In [42]:
item_prediction = predict(train, item_similarity, kind='item')
user_prediction = predict(train, user_similarity, kind='user')
print('User-based CF MSE: ' + str(get_mse(user_prediction, test)))
print('Movie-based CF MSE: ' + str(get_mse(item_prediction, test)))

User-based CF MSE: 9.09635971233
Movie-based CF MSE: 12.0976455214


In [43]:
df_Third = pd.read_csv("/Users/sushma/Desktop/DM_Git/Recommender_System/ThirdRatingsMatrix.csv", header=0)
df_Third.head()

,UserID,MovieID,Rating,label
0,1,3408,4,2
1,1,1287,5,2
2,1,527,5,2
3,1,1097,4,2
4,1,1545,4,2


In [44]:
uniq_userid = df_Third.UserID.unique()
sorted_uniq_userid = sorted(uniq_userid)
uindex = 0
user_index = {}
for n in sorted_uniq_userid:
    user_index[n] = uindex
    uindex += 1
uniq_movieid = df_Third.MovieID.unique()
sorted_uniq_movieid = sorted(uniq_movieid)
mindex = 0
movie_index = {}
for n in sorted_uniq_movieid:
    movie_index[n] = mindex
    mindex += 1

In [45]:
n_users = df_Third.UserID.unique().shape[0]
print("n users: %s" % n_users)
n_movies = df_Third.MovieID.unique().shape[0]
print("n movies: %s" % n_movies)

n users: 5981
n movies: 685


In [46]:
third_clust_ratings = np.zeros((n_users,n_movies))
for row in df_Third.itertuples():
    # row[1] will the user_id; row[2] the 
    # rating (see also my note below).
    # the -1 2s to move to 0 indexing.
    third_clust_ratings[user_index[row[1]]-1, movie_index[row[2]]-1] = row[3]
third_clust_ratings

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  3.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [47]:
sparsity = float(len(third_clust_ratings.nonzero()[0]))
sparsity /= (third_clust_ratings.shape[0] * third_clust_ratings.shape[1])
sparsity *= 100
print('sparsity: {:4.2f}%'.format(sparsity))

sparsity: 4.06%


In [48]:
train, test = train_test_split(third_clust_ratings)

In [49]:
user_similarity = similarity(train, kind='user')
item_similarity = similarity(train, kind='item')

[[  4.79000000e+02   2.00000000e+01   4.50000000e+01 ...,   2.00000000e+01
    2.28000000e+02   3.60000000e+01]
 [  2.00000000e+01   6.40000000e+01   8.00000000e+00 ...,   1.00000000e-09
    4.40000000e+01   1.00000000e-09]
 [  4.50000000e+01   8.00000000e+00   7.00000000e+01 ...,   1.60000000e+01
    4.60000000e+01   1.00000000e-09]
 ..., 
 [  2.00000000e+01   1.00000000e-09   1.60000000e+01 ...,   7.30000000e+01
    2.00000000e+01   1.00000000e-09]
 [  2.28000000e+02   4.40000000e+01   4.60000000e+01 ...,   2.00000000e+01
    1.29300000e+03   2.80000000e+01]
 [  3.60000000e+01   1.00000000e-09   1.00000000e-09 ...,   1.00000000e-09
    2.80000000e+01   8.00000000e+01]]


In [51]:
sim = third_clust_ratings.dot(train.T) + 1e-9
sim

array([[  4.79000000e+02,   4.00000000e+01,   5.50000000e+01, ...,
          2.00000000e+01,   2.73000000e+02,   3.60000000e+01],
       [  2.00000000e+01,   6.40000000e+01,   8.00000000e+00, ...,
          1.00000000e-09,   4.40000000e+01,   1.00000000e-09],
       [  4.50000000e+01,   8.00000000e+00,   7.00000000e+01, ...,
          1.60000000e+01,   4.60000000e+01,   1.00000000e-09],
       ..., 
       [  4.00000000e+01,   1.00000000e-09,   1.60000000e+01, ...,
          7.30000000e+01,   7.60000000e+01,   1.00000000e-09],
       [  2.52000000e+02,   4.40000000e+01,   4.60000000e+01, ...,
          2.00000000e+01,   1.29300000e+03,   2.80000000e+01],
       [  7.60000000e+01,   2.00000000e+01,   4.10000000e+01, ...,
          1.60000000e+01,   1.10000000e+02,   8.00000000e+01]])

In [52]:
item_prediction = predict(train, item_similarity, kind='item')
user_prediction = predict(train, user_similarity, kind='user')
print('User-based CF MSE: ' + str(get_mse(user_prediction, test)))
print('Movie-based CF MSE: ' + str(get_mse(item_prediction, test)))

User-based CF MSE: 10.764265726
Movie-based CF MSE: 14.0112636152
